# Praca domowa 2
#### Łukasz Tomaszewski

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('hw1.csv')
df = df.drop(['Unnamed: 0'], axis =1 )

In [3]:
y = df['readmitted']
X = df.drop(['readmitted'], axis = 1)

Podzielmy ramkę na zbiór treningowy i testowy.

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

Wybierzmy model - RandomForestClassifier.

In [6]:
from sklearn.ensemble import RandomForestClassifier

## Domyślne parametry

In [7]:
rfc_clf = RandomForestClassifier()
rfc_clf.fit(X_train, y_train)

RandomForestClassifier()

In [8]:
predicted_class = rfc_clf.predict(X_test)
predicted_prob = rfc_clf.predict_proba(X_test)

Jako drugi sposób walidacji wybieramy accuracy_score.

In [9]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [10]:
roc_auc_score(y_test, predicted_prob[:,1])

0.8441008043128087

In [11]:
accuracy_score(y_test, predicted_class)

0.7788659962658456

Dla AUC udało nam sie osiągnąc wynik 0.844, co jest całkiem dobrym rezultatem. Spróbujmy jeszcze polepszyć otrzymany wynik.

## Ręczna zmiana hiperparametrów

Parametrami, które możemy poprawić wydają się być parametry: n_estimators, criterion, min_samples_leaf oraz min_samples_split. Przetestujmy różne kombinację wartości dla tych parametrów.
(Poniższe parametry są efektem wielu prób, z których zostały wybrane najlepsze wartości)

In [12]:
rfc_clf = RandomForestClassifier(n_estimators = 1500,criterion='entropy', min_samples_leaf = 2,min_samples_split = 4,
                                 n_jobs=-1)
rfc_clf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', min_samples_leaf=2,
                       min_samples_split=4, n_estimators=1500, n_jobs=-1)

Dla wybranych parametrów runtime trwał dłużej, jednak dał lepsze efekty niż dla parametórw domyślnych. Wydaje się, że najważniejszym parametrem jest n_estimators - jego zwiększenie najbardziej poprawia rezultat, jednak powyżej wartości 1500 rezultat był bardzo zbliżony, a nawet gorszy, a czas potrzebny na wykonanie znacząco wzrastał.

In [13]:
predicted_class = rfc_clf.predict(X_test)
predicted_prob = rfc_clf.predict_proba(X_test)

In [14]:
roc_auc_score(y_test, predicted_prob[:,1])

0.8508373369945336

In [15]:
accuracy_score(y_test, predicted_class)

0.7804055160666907

Dzięki ręcznej zmianie hiperparametrów udało nam się uzyskać nieco lepszy rezultat niż ten dla ustawień domyślnych. Spróbujmy znaleźć lepsze parametry, wykorzystując poniższe funkcje. 
Dla każdej z nich cv = 3

## Random Search

In [16]:
from sklearn.model_selection import RandomizedSearchCV

In [17]:
rfc_clf = RandomForestClassifier()

In [18]:
import skopt, scipy

Wylosujmy 50 przypadków, gdzie możliwe wartości parametrów opisane są poniżej.

In [19]:
parameters = {
    'criterion': ['gini', 'entropy'],
    'min_samples_leaf': scipy.stats.randint(2,20),
    'min_samples_split': scipy.stats.randint(3,20),
    'n_estimators': scipy.stats.randint(100,1500)
}

rs = RandomizedSearchCV(estimator = rfc_clf, param_distributions = parameters, verbose=10, 
                               cv = 3, n_iter = 50, n_jobs = -1, scoring='roc_auc')
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026381BEE640>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026381BEE280>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026381BEE3D0>},
                   scoring='roc_auc', verbose=10)

Jakie były najlepsze parametry:

In [20]:
rs.best_params_

{'criterion': 'entropy',
 'min_samples_leaf': 2,
 'min_samples_split': 11,
 'n_estimators': 742}

Weźmy najlepsze parametry i sprawdźmy dla nich AUC.

In [21]:
rs = rs.best_estimator_

In [22]:
predicted_class = rs.predict(X_test)
predicted_prob = rs.predict_proba(X_test)

In [23]:
roc_auc_score(y_test, predicted_prob[:,1])

0.8511484391511555

In [24]:
accuracy_score(y_test, predicted_class)

0.7802744931049166

Udało nam się osiągnąć wynik lepszy niż w przypadku ręcznej zmiany hiperparametrów.

## Grid Search

In [25]:
from sklearn.model_selection import GridSearchCV

In [26]:
rfc_clf = RandomForestClassifier()

Sprawdźmy, która kombinacja podanych niżej hiperparametrów jest najlepsza.

In [27]:
parameters = {
    'criterion': ['gini', 'entropy'],
    'min_samples_leaf': [2, 3, 5, 10],
    'min_samples_split': [3, 4, 6, 10],
    'n_estimators': [500, 1000]
}

gs = GridSearchCV(estimator = rfc_clf, param_grid = parameters, n_jobs = -1, verbose = 10, cv = 3,
                  scoring='roc_auc')
gs.fit(X_train, y_train)

Fitting 3 folds for each of 64 candidates, totalling 192 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'min_samples_leaf': [2, 3, 5, 10],
                         'min_samples_split': [3, 4, 6, 10],
                         'n_estimators': [500, 1000]},
             scoring='roc_auc', verbose=10)

Najlepsza kombinacja:

In [28]:
gs.best_params_

{'criterion': 'entropy',
 'min_samples_leaf': 3,
 'min_samples_split': 10,
 'n_estimators': 1000}

Sprawdźmy AUC.

In [29]:
gs = gs.best_estimator_

In [30]:
predicted_class = gs.predict(X_test)
predicted_prob = gs.predict_proba(X_test)

In [31]:
roc_auc_score(y_test, predicted_prob[:,1])

0.8508267230021713

In [32]:
accuracy_score(y_test, predicted_class)

0.7800124471813685

Wynik nieznacznie gorszy od wyniku uzyskanego przy ręcznym wyborze hiperparametrów.

## Optymalizacja bayesowska

In [33]:
rfc_clf = RandomForestClassifier()

Wylosujmy 50 przypadków, gdzie możliwe wartości parametrów opisane są poniżej.

In [34]:
search_spaces = {
        'criterion': ['gini', 'entropy'],
        'min_samples_split': skopt.space.space.Integer(2, 20),
        'min_samples_leaf': skopt.space.space.Integer(3, 20),
        'n_estimators': skopt.space.space.Integer(100,1500)
}

In [35]:
ob = skopt.BayesSearchCV(
        estimator=rfc_clf,
        search_spaces=search_spaces,
        scoring='roc_auc',
        n_jobs=-1,
        n_iter = 50,
        verbose= 10,
        cv = 3
    )
ob.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

BayesSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
              scoring='roc_auc',
              search_spaces={'criterion': ['gini', 'entropy'],
                             'min_samples_leaf': Integer(low=3, high=20, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=2, high=20, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=100, high=1500, prior='uniform', transform='normalize')},
              verbose=10)

Najlepsze znalezione parametry:

In [36]:
ob.best_params_

OrderedDict([('criterion', 'entropy'),
             ('min_samples_leaf', 3),
             ('min_samples_split', 8),
             ('n_estimators', 1359)])

Sprawdźmy wyniki.

In [37]:
ob = ob.best_estimator_

In [38]:
predicted_class = ob.predict(X_test)
predicted_prob = ob.predict_proba(X_test)

In [39]:
roc_auc_score(y_test, predicted_prob[:,1])

0.850817036655483

In [40]:
accuracy_score(y_test, predicted_class)

0.779979691440925

Optymalizacja bayesowska uzyskała wynik zbliżony do Grid Searcha.

## Podsumowanie

Wszystkie metody, które zmieniały hiperparametry okazały się byś lepsze niż domyślna funkcja. Najlepiej zadziałał RandomSearch, następnie była ręczna zmiana hiperparametrów, GridSearch i optymalizacja bayesowska. Różnice w wyniku dla sposobów modyfikujących hiperparamtery nie były jednak duże.